In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from feature_extractor import loadFeatureExtractor
#from torchsummary import summary
%load_ext tensorboard

In [ ]:
model = loadFeatureExtractor()
x = torch.randn((6, 256, 256)).unsqueeze(0)
y = model(x)

In [ ]:
#summary(model, (6,256,256))

In [ ]:
y.shape

In [ ]:
y.flatten().shape

In [ ]:
writer = SummaryWriter()
writer.add_graph(model, x)
writer.close()

In [ ]:
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

---

In [4]:
from pathlib import Path

folders = list(Path("C:/xBD/test_bldgs/").glob('*'))

In [10]:
import pandas as pd
df = pd.read_csv(list(folders[0].glob('*.csv'))[0], index_col=0)
df.head()

,coords,class
guatemala-volcano_00000003_post_disaster_0.png,"(465.94617850181066, 109.62942748980328)",minor-damage
guatemala-volcano_00000003_post_disaster_1.png,"(240.50073856928046, 314.8236608385315)",destroyed
guatemala-volcano_00000003_post_disaster_2.png,"(591.9558061636591, 266.25772937220216)",minor-damage
guatemala-volcano_00000005_post_disaster_0.png,"(814.0503282467604, 958.26675138958)",un-classified
guatemala-volcano_00000009_post_disaster_0.png,"(319.7156849146396, 745.2364736911228)",un-classified


In [11]:
pre = list(folders[0].glob('*pre_disaster*'))

In [12]:
post = list(folders[0].glob('*post_disaster*'))

In [21]:
pre[4]

WindowsPath('C:/xBD/test_bldgs/guatemala-volcano/guatemala-volcano_00000009_pre_disaster_0.png')

In [22]:
post[4]

WindowsPath('C:/xBD/test_bldgs/guatemala-volcano/guatemala-volcano_00000009_post_disaster_0.png')